In [1]:
import pandas as pd
from unidecode import unidecode

# reading the kaggle dataset
kaggle_data = pd.read_csv('../ufcdata/data.csv')

# reading the elevations dataset(scraped)
elevations_data = pd.read_csv('../generated_data/locations_elevation_updated.csv')

In [2]:
# fixing one location entry in the kaggle dataset
kaggle_data.loc[kaggle_data['location'] == 'Singapore, Singapore', 'location'] = 'Marina Bay, Singapore'

def cleanLocationsForJoin(strs):
    words = strs.split(',')
    return (', '.join([unidecode(words[0].strip()), unidecode(words[-1].strip())]))

# preparing data for joining
kaggle_data['merge_location'] = kaggle_data['location'].apply(cleanLocationsForJoin)
elevations_data['merge_location'] = elevations_data['location'].apply(cleanLocationsForJoin)

#dropping the original location column from elevations dataset by keeping in kaggle dataset to use later
elevations_data.drop(columns=['location'], inplace=True)

# removing meter(m) abbr. from the elevation column and changing data type to float
elevations_data['location_elevation'] = elevations_data['location_elevation'].apply(lambda x: float(x.split()[0]))

In [3]:
# printing the basic statistics of both datasets
def printColumnStats(dataset):
    for column in dataset.columns:
        print('{:30s}'.format(column), dataset[column].dtype)

# shape of kaggle dataset
print('Kaggle dataset shape:', '{}\n'.format(kaggle_data.shape))
printColumnStats(kaggle_data)
print('\n\n')

# shape of elevations dataset
print('Elevations dataset shape:', '{}\n'.format(elevations_data.shape))
printColumnStats(elevations_data)

Kaggle dataset shape: (5144, 146)

R_fighter                      object
B_fighter                      object
Referee                        object
date                           object
location                       object
Winner                         object
title_bout                     bool
weight_class                   object
no_of_rounds                   int64
B_current_lose_streak          float64
B_current_win_streak           float64
B_draw                         float64
B_avg_BODY_att                 float64
B_avg_BODY_landed              float64
B_avg_CLINCH_att               float64
B_avg_CLINCH_landed            float64
B_avg_DISTANCE_att             float64
B_avg_DISTANCE_landed          float64
B_avg_GROUND_att               float64
B_avg_GROUND_landed            float64
B_avg_HEAD_att                 float64
B_avg_HEAD_landed              float64
B_avg_KD                       float64
B_avg_LEG_att                  float64
B_avg_LEG_landed               float64
B_

In [4]:
# join the two columns to add the elevations for locations
joined_data = kaggle_data.set_index('merge_location').join(elevations_data.set_index('merge_location')).reset_index()

#dropping the merge location column from joined dataset, the location is under location column
joined_data.drop(columns=['merge_location'], inplace=True)

# printing the stats of the newly joined dataset
print('Joined dataset shape:', '{}\n'.format(joined_data.shape))
printColumnStats(joined_data)

Joined dataset shape: (5144, 146)

R_fighter                      object
B_fighter                      object
Referee                        object
date                           object
location                       object
Winner                         object
title_bout                     bool
weight_class                   object
no_of_rounds                   int64
B_current_lose_streak          float64
B_current_win_streak           float64
B_draw                         float64
B_avg_BODY_att                 float64
B_avg_BODY_landed              float64
B_avg_CLINCH_att               float64
B_avg_CLINCH_landed            float64
B_avg_DISTANCE_att             float64
B_avg_DISTANCE_landed          float64
B_avg_GROUND_att               float64
B_avg_GROUND_landed            float64
B_avg_HEAD_att                 float64
B_avg_HEAD_landed              float64
B_avg_KD                       float64
B_avg_LEG_att                  float64
B_avg_LEG_landed               float64
B_

In [5]:
# checking the integrity of the joined location elevations data
print('Empty Location Values:', joined_data['location_elevation'].isnull().any())

Empty Location Values: False


In [6]:
joined_data.head()

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,location_elevation
0,Minotauro Nogueira,Roy Nelson,Leon Roberts,2014-04-11,"Abu Dhabi, United Arab Emirates",Blue,False,Heavyweight,5,2.0,...,2.0,0.0,5.0,Orthodox,190.50,195.58,240.0,37.0,37.0,27.0
1,Clay Guida,Tatsuya Kawajiri,Marc Goddard,2014-04-11,"Abu Dhabi, United Arab Emirates",Red,False,Featherweight,3,0.0,...,4.0,0.0,10.0,Orthodox,170.18,177.80,155.0,35.0,32.0,27.0
2,John Howard,Ryan LaFlare,Leon Roberts,2014-04-11,"Abu Dhabi, United Arab Emirates",Blue,False,Welterweight,3,0.0,...,0.0,0.0,6.0,Orthodox,170.18,180.34,170.0,30.0,31.0,27.0
3,Ramsey Nijem,Beneil Dariush,Neil Hall,2014-04-11,"Abu Dhabi, United Arab Emirates",Red,False,Lightweight,3,0.0,...,0.0,0.0,4.0,Orthodox,180.34,190.50,155.0,24.0,26.0,27.0
4,Jared Rosholt,Daniel Omielanczuk,Marc Goddard,2014-04-11,"Abu Dhabi, United Arab Emirates",Red,False,Heavyweight,3,0.0,...,0.0,0.0,1.0,Orthodox,187.96,190.50,265.0,31.0,27.0,27.0


In [7]:
# creating a new csv
joined_data.to_csv('../ufcdata/data_with_location_elevation.csv', index=False)